In [1]:
import os

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx', 'crawler_2.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/2.part/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Part Number'].tolist()

总数量：10


['40-1000',
 '40-10000',
 '40-10005',
 '40-10006',
 '40-10007',
 '40-10009',
 '40-1001',
 '40-10013',
 '40-10014',
 '40-10015']

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

b = 0
while True:
    b += 1
    print(b)
    try:
        request_url, check_url = f'''https://productdesk-api.cellacore.net/legacy/cardone/FinishedGood/?q=productDeskItemNumber.keyword:{input_.loc[a, 'Part Number']}&size=1&useCache=true&auth={input_.loc[a, 'Auth']}''', input_.loc[a, 'Url']
        
        resp = requests.get(request_url,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200:
            if resp.json():
                dict_ = resp.json()[0]
                break
    except KeyboardInterrupt:
        break
    except:
        continue

dict_

1


{'__type': 'FinishedGood',
 '__isSchema': False,
 '__path': '/ProductDeskEntity/',
 '__fullPath': '/ProductDeskEntity/CPCX|40-1000',
 '__modified': '2024-09-24T08:00:05.090Z',
 '__instanceId': 'CPCX|40-1000',
 '__legacyId': '2085510957217265418',
 'productDeskName': 'Windshield Wiper Motor',
 'id': 'CPCX|40-1000',
 'RelatedProducts': [],
 'modifiedOn': '2024-09-24T08:00:05.090Z',
 'channels': [{'code': 'R0164',
   'name': 'B Attributes',
   'requiresApproval': 0,
   'id': '5500392672583901'},
  {'code': 'R0159',
   'name': '"Popularity Sorting"',
   'requiresApproval': 0,
   'id': '14085640583406600'},
  {'code': 'R0118',
   'name': 'All SKU only',
   'requiresApproval': 0,
   'id': '79040821002085808'},
  {'code': 'R0161',
   'name': 'Show on Site',
   'requiresApproval': 0,
   'id': '115580352379583008'},
  {'code': 'R0107',
   'name': 'Vibrant Marketing Q',
   'requiresApproval': 0,
   'id': '125106138327532704'},
  {'code': 'R0155',
   'name': '"Descriptions"',
   'requiresApproval

In [8]:
list_oem, list_dict = [], dict_['CrossReferences'] if 'CrossReferences' in dict_ else []
for dict_ in list_dict:
    if dict_['referenceType'].strip() == 'OE Part':
        list_oem.append(dict_['reference'].strip())

oem = ';'.join(sorted(list(set(list_oem)), key=list_oem.index))

oem

'22154913;22110039'

In [9]:
df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Type': input_.loc[a, 'Type'],
                         'Part Number': input_.loc[a, 'Part Number'],
                         'Title': input_.loc[a, 'Title'],
                         'Subtitle': input_.loc[a, 'Subtitle'],
                         'Application Summary': input_.loc[a, 'Application Summary'],
                         'OEM': oem,
                         'Picture': input_.loc[a, 'Picture'],
                         'Url': input_.loc[a, 'Url'],
                         'Json_Src': input_.loc[a, 'Json_Src'],
                         'Json_Info': input_.loc[a, 'Json_Info'],
                         'Bundle': input_.loc[a, 'Bundle'],
                         'Auth': input_.loc[a, 'Auth']}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Type,Part Number,Title,Subtitle,Application Summary,OEM,Picture,Url,Json_Src,Json_Info,Bundle,Auth
0,1,Windshield Wiper Motor,40-1000,Windshield Wiper Motor - 40-1000,CARDONE Reman,Chevrolet Camaro 1998-93; Pontiac Firebird 199...,22154913;22110039,,https://www.cardone.com/product/windshield-wip...,"{""0"": ""https://cdn11.bigcommerce.com/s-z9vhe3o...","{""0"": {""productDeskExternalKey"": ""CPCX|40-1000...",https://cdn11.bigcommerce.com/s-z9vhe3o238/ste...,mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw


In [10]:
crawler_status = 'ok'

crawler_status

'ok'

In [11]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url
    df_temp.loc[0, 'Check_Url'] = check_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
    
output_error

""


In [12]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_oem.xlsx', index=False)

output_correct

,No,Type,Part Number,Title,Subtitle,Application Summary,OEM,Picture,Url,Json_Src,Json_Info,Bundle,Auth
0,1,Windshield Wiper Motor,40-1000,Windshield Wiper Motor - 40-1000,CARDONE Reman,Chevrolet Camaro 1998-93; Pontiac Firebird 199...,22154913;22110039,,https://www.cardone.com/product/windshield-wip...,"{""0"": ""https://cdn11.bigcommerce.com/s-z9vhe3o...","{""0"": {""productDeskExternalKey"": ""CPCX|40-1000...",https://cdn11.bigcommerce.com/s-z9vhe3o238/ste...,mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw


In [13]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_oem_error.xlsx', index=False)

output_error

""
